In [1]:
# group by 技术
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import matplotlib.pyplot as plt
from numpy.random import randn
import os
from datetime import datetime

In [2]:
df = DataFrame({'key1':['a','a','b','b','a'],
                'key2':['one','two','one','two','one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.886721,0.552640
1,a,two,-1.696925,2.192593
2,b,one,-0.027817,0.927517
3,b,two,0.770397,1.127878
4,a,one,-0.377522,1.044947


In [3]:
grouped = df['data1'].groupby(df['key1'])   # 访问data1根据key1调用group by
grouped

In [4]:
grouped.mean()

key1
a   -0.987056
b    0.371290
Name: data1, dtype: float64

In [7]:
# 如果groupby 的时候一次传入多个数组，那么情况就会不一样了
means = df['data1'].groupby([df['key1'],df['key2']]).mean()   # 按照两个key索引会得到一个层次化索引的数据（由唯一的键对组成）
means

key1  key2
a     one    -0.632122
      two    -1.696925
b     one    -0.027817
      two     0.770397
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,-0.632122,-1.696925
b,-0.027817,0.770397
